In [63]:
import os
from spatial_graphs.AmiraSpatialGraph import AmiraSpatialGraph
from spatial_graphs.Landmarks import Landmarks
from spatial_graphs.Surfaces import Surface
from spatial_graphs.Vectors import Vectors
from spatial_graphs.Alignment import Alignment
from spatial_graphs.Stats import Stats

In [10]:
def create_surface(file,coronal=False,rhs=False,nr_divisions=0,surf_resolution='200'):
    print(file)
    #print(nr_divisions)
    os.system('/home/mythreya/project_src/BarrelField3D/DataAnalysis3D/BF3DRecon {} {} {}'.\
              format(file,input_path+os.path.basename(file)[:-3],str(surf_resolution)))
    
    

In [9]:
input_path = '/nas1/Data_Mythreya/MotorCortexProject/'

In [78]:
lhs = AmiraSpatialGraph(input_path+'atlas_20210217_Pia_lhs.am')
lhs.pia.set_label('9')
lhs.update_graph_data()
final_sg = AmiraSpatialGraph()
final_sg.graph_data = final_sg.combine_subgraphs([final_sg.graph_data,lhs.pia,])
final_sg.update_graph_data()
final_sg.write_sub_spatial_graph(lhs.pia,input_path+'atlas_20210217_Pia_lhs_relabelled.am')

rhs = AmiraSpatialGraph(input_path+'atlas_20210217_Pia_rhs.am')
rhs.pia.set_label('9')
rhs.update_graph_data()
final_sg = AmiraSpatialGraph()
final_sg.graph_data = final_sg.combine_subgraphs([final_sg.graph_data,rhs.pia,])
final_sg.update_graph_data()
final_sg.write_sub_spatial_graph(rhs.pia,input_path+'atlas_20210217_Pia_rhs_relabelled.am')

In [105]:
sg = AmiraSpatialGraph(input_path+'atlas_20210217_Pia_lhs_relabelled.am')
lhs_hull = Surface(pts=sg.graph_data.edge_pt_coords).create_delunay_surface_3d(output_filename=input_path+'atlas_lhs.vtk',\
                                                                               return_hull=True)
lhs_surf = Surface(polydata=lhs_hull)
lhs_surf.clip_surface_at_given_x(0,x_limit=lhs_hull.GetBounds()[1]-400)
lhs_surf.write_surface_mesh(input_path+'atlas_lhs_open.vtk')

sg = AmiraSpatialGraph(input_path+'atlas_20210217_Pia_rhs_relabelled.am')
rhs_hull = Surface(pts=sg.graph_data.edge_pt_coords).create_delunay_surface_3d(output_filename=input_path+'atlas_rhs.vtk',\
                                                                               return_hull=True)
rhs_surf = Surface(polydata=rhs_hull)
rhs_surf.clip_surface_at_given_x(0,x_limit=rhs_hull.GetBounds()[0]+400)
rhs_surf.write_surface_mesh(input_path+'atlas_rhs_open.vtk')

In [106]:
tr_mat = [-1, 0, 0, 0, 0, 0, 1, 0, 0, -1, 0, 0, 0, 0, 0, 1]
lhs_surf.apply_transformation(tr_mat)
lhs_surf.write_surface_mesh(input_path+'atlas_rhs_canonical.vtk')
rhs_surf.apply_transformation(tr_mat)
rhs_surf.write_surface_mesh(input_path+'atlas_lhs_canonical.vtk')

In [107]:
# Align to the avg surfaces

In [116]:
avg_path = '/nas1/Data_Mythreya/MotorCortexProject/V11/vM1_Ref_Frame/Outputs/Alignment/Tangential_To_Coronal/Average/'

In [117]:
avg_lhs_pia = Surface(avg_path+'LHS/Avg_Pia_200.vtk')
avg_rhs_pia = Surface(avg_path+'RHS/Avg_Pia_200.vtk')

In [118]:
lhs_surf = Surface(input_path+'atlas_lhs_canonical.vtk')
aligner = Alignment(avg_lhs_pia.surface,lhs_surf.surface,)
txmat = aligner.get_transformation_matrix()
icp = aligner.get_icp_transform()
lhs_surf.apply_icp_transform(icp)
lhs_surf.write_surface_mesh(input_path+'atlas_lhs_aligned_to_avg.vtk')
aligner.write_transformation_matrix(input_path+'atlas_canonical_to_avg_transformation_lhs.txt')

In [119]:
lhs_surf = Surface(input_path+'atlas_lhs_canonical.vtk')
aligner = Alignment(avg_lhs_pia.surface,lhs_surf.surface,mode='similarity')
txmat = aligner.get_transformation_matrix()
icp = aligner.get_icp_transform()
lhs_surf.apply_icp_transform(icp)
lhs_surf.write_surface_mesh(input_path+'atlas_lhs_aligned_to_avg_scaling.vtk')
aligner.write_transformation_matrix(input_path+'atlas_canonical_to_avg_transformation_lhs_scaling.txt')

In [120]:
rhs_surf = Surface(input_path+'atlas_rhs_canonical.vtk')
aligner = Alignment(avg_rhs_pia.surface,rhs_surf.surface)
txmat = aligner.get_transformation_matrix()
icp = aligner.get_icp_transform()
rhs_surf.apply_icp_transform(icp)
rhs_surf.write_surface_mesh(input_path+'atlas_rhs_aligned_to_avg.vtk')
aligner.write_transformation_matrix(input_path+'atlas_canonical_to_avg_transformation_rhs.txt')

In [121]:
rhs_surf = Surface(input_path+'atlas_rhs_canonical.vtk')
aligner = Alignment(avg_rhs_pia.surface,rhs_surf.surface,mode='similarity')
txmat = aligner.get_transformation_matrix()
icp = aligner.get_icp_transform()
rhs_surf.apply_icp_transform(icp)
rhs_surf.write_surface_mesh(input_path+'atlas_rhs_aligned_to_avg_scaling.vtk')
aligner.write_transformation_matrix(input_path+'atlas_canonical_to_avg_transformation_rhs_scaling.txt')